# TP 7 - Méthodes d'ensembles 

Les données sont celles des derniers cours (prix des maisons) et sont disponible sur mon site ou sur kaggle, en cas de besoin.

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


### Fast Preprocessing

In [2]:
data = pd.read_csv('data.csv')

In [3]:
# Suppression des outliers

data = data.drop(data[(data['GrLivArea']>4000) & (data['SalePrice']<300000)].index)

In [4]:
# Complétion des valeurs manquantes
data[list(data.select_dtypes(object).columns)] = data[list(data.select_dtypes(object).columns)].fillna('__NC__')
data[list(data.select_dtypes(float).columns)] = data[list(data.select_dtypes(float).columns)].apply(lambda x: x.fillna(x.mean()),axis=0)
data[list(data.select_dtypes(int).columns)] = data[list(data.select_dtypes(int).columns)].apply(lambda x: x.fillna(x.value_counts().index[0]),axis=0)

In [5]:
for c in data.select_dtypes(object).columns:
    data[c] = LabelEncoder().fit_transform(data[c])

In [6]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,3,65.0,8450,1,2,3,3,0,...,0,3,4,4,0,2,2008,8,4,208500
1,2,20,3,80.0,9600,1,2,3,3,0,...,0,3,4,4,0,5,2007,8,4,181500
2,3,60,3,68.0,11250,1,2,0,3,0,...,0,3,4,4,0,9,2008,8,4,223500
3,4,70,3,60.0,9550,1,2,0,3,0,...,0,3,4,4,0,2,2006,8,0,140000
4,5,60,3,84.0,14260,1,2,0,3,0,...,0,3,4,4,0,12,2008,8,4,250000


### Séparation des données

In [7]:
y = data.SalePrice
data = data.drop(['Id','SalePrice'], axis = 1)

In [8]:
# Séparatation en trois jeux de données
X_train, X_tmp, y_train, y_tmp = train_test_split(data, y, test_size = 0.4, random_state = 2019)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size = 0.5, random_state = 2019)

In [9]:
print('Train shape {}'.format(X_train.shape))
print('Validation shape {}'.format(X_val.shape))
print('Test shape {}'.format(X_test.shape))

Train shape (874, 79)
Validation shape (292, 79)
Test shape (292, 79)


In [10]:
def rmse(preds, targets):
    return np.sqrt(((preds - targets) ** 2).mean())

**1** Evaluer les performances, avec la rmse, de cinqs modèles différents au choix sur le jeu de test. 
Stocker les prédictions de chaque modèle dans un dictionnaire "preds" de format suivant :

preds = dict()

preds['model1'] = model1.predict(X_test)

In [11]:
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor,GradientBoostingRegressor


In [19]:
model1 = XGBRegressor()
model1.fit(X_train, y_train)
model2 = RandomForestRegressor()
model2.fit(X_train, y_train)
model3 = ExtraTreesRegressor()
model3.fit(X_train, y_train)
model4 = GradientBoostingRegressor()
model4.fit(X_train, y_train)
model5 = Ridge()
model5.fit(X_train, y_train)


/Users/arthur/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[18:26:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/arthur/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/arthur/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [20]:
preds=dict()
preds['model1']=model1.predict(X_test)
preds['model2']=model2.predict(X_test)
preds['model3']=model3.predict(X_test)
preds['model4']=model4.predict(X_test)
preds['model5']=model5.predict(X_test)


In [21]:
for key in preds.keys():
    print(f'{key} rmse {rmse(y_test, preds[key])}')

model1 rmse 25678.725448485853
model2 rmse 27538.657587597696
model3 rmse 26863.73300773663
model4 rmse 24355.13581859164
model5 rmse 26723.9404066337


In [12]:
regs = {}
regs['rdg'] = {'reg':Ridge(),'name':'Ridge'}
regs['xgb'] = {'reg':XGBRegressor(n_estimators=150,random_state= 0),'name':'XGBRegressor'}
regs['ext'] = {'reg':ExtraTreesRegressor(n_estimators=150,random_state= 0),'name':'ExtraTreesRegressor'}
regs['rf'] = {'reg':RandomForestRegressor(n_estimators=150,random_state= 0),'name':'RandomForestRegressor'}
regs['gbt'] = {'reg':GradientBoostingRegressor(n_estimators=150,random_state= 0),'name':'GradientBoostingRegressor'}


preds = {}

for key in regs:
    regs[key]['reg'].fit(X_train,y_train)
    preds[key] = regs[key]['reg'].predict(X_test)
    print ('{} a une performance de {}'.format(regs[key]['name'],rmse(y_test,preds[key])))
    

Ridge a une performance de 26723.9404066337
[18:23:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/arthur/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor a une performance de 25267.225442100796
ExtraTreesRegressor a une performance de 24964.81789682046
RandomForestRegressor a une performance de 26533.167552363208
GradientBoostingRegressor a une performance de 24241.46155717974


**2** Améliorer les performances en aggrégant par moyenne et médiane.

In [13]:
#Il suffit d'utiliser les prédictions stockées:
print ('Agregation par moyenne : {}'.format(rmse(y_test,np.mean(list(preds.values()),axis=0))))
print ('Agregation par mediane : {}'.format(rmse(y_test,np.median(list(preds.values()),axis=0))))


Agregation par moyenne : 23521.703491886074
Agregation par mediane : 23995.415588553733


**3** Améliorer les performances en utilisant une agrégation pondérée de vos modèles. Essayez plusieurs poids, et chercher les plus performants.
Comment faire pour trouver des poids optimaux ?

In [14]:
prediction = preds['rf']*0.2 + preds['ext']*0.1 + preds['gbt']*0.2 + preds['xgb']*0.35 + preds['rdg']*0.15
print ('Agregation pondérée : {}'.format(rmse(y_test,prediction))) # Je met des poids dans l'ordre de performance

prediction = preds['rf']*0.15+ preds['ext']*0.05 + preds['gbt']*0.15 + preds['xgb']*0.5 + preds['rdg']*0.15
print ('Agregation pondérée : {}'.format(rmse(y_test,prediction))) #Je favorise le meilleur modèle


Agregation pondérée : 23788.47737690504
Agregation pondérée : 23970.103376916308


Il suffit de définir une fonction objectif comme l'aggrégation pondéré des erreurs et trouver une manière de la minimiser ! Bayésien, SGD..

**4** Améliorer les performances de deux machines grâce la méthode de bagging.

In [15]:
bags = 10
preds = []
for i in range(bags):
    model = RandomForestRegressor(n_estimators=200,random_state=10)
    model.fit(X_train,y_train)
    preds.append(model.predict(X_test))

print ('Bagging RF: {}'.format(rmse(y_test,np.mean(preds,axis=0))))


Bagging RF: 26484.10059468568


In [16]:
bags = 10
preds = []
for i in range(bags):
    model = GradientBoostingRegressor(n_estimators=200,random_state=10)
    model.fit(X_train,y_train)
    preds.append(model.predict(X_test))
    
print ('Bagging GBT: {}'.format(rmse(y_test,np.mean(preds,axis=0))))


Bagging GBT: 24059.79623011579


**5** Implémentez une méthode d'apprentissage sur 5 Folds pour un modèle au choix.

In [17]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, random_state=42)

pred = 0
for i, (tr_idx, val_idx) in enumerate(kfold.split(X_train)):
    model = XGBRegressor(n_estimators=1000)
    model.fit(X_train.iloc[tr_idx],y_train.iloc[tr_idx], eval_set = [(X_train.iloc[val_idx],y_train.iloc[val_idx])], early_stopping_rounds= 50, verbose=100)
    pred+=model.predict(X_test) / 5.

[0]	validation_0-rmse:186648
Will train until validation_0-rmse hasn't improved in 50 rounds.
[100]	validation_0-rmse:22382.9
[200]	validation_0-rmse:21993.8
[300]	validation_0-rmse:21792.3
[400]	validation_0-rmse:21595.2
[500]	validation_0-rmse:21426.6
[600]	validation_0-rmse:21238.2
[700]	validation_0-rmse:21172
Stopping. Best iteration:
[712]	validation_0-rmse:21167.5

[0]	validation_0-rmse:177467
Will train until validation_0-rmse hasn't improved in 50 rounds.
[100]	validation_0-rmse:27922
[200]	validation_0-rmse:26986.3
[300]	validation_0-rmse:26787
Stopping. Best iteration:
[274]	validation_0-rmse:26708.2

[0]	validation_0-rmse:177980
Will train until validation_0-rmse hasn't improved in 50 rounds.
[100]	validation_0-rmse:28563.3
[200]	validation_0-rmse:27839.3
Stopping. Best iteration:
[217]	validation_0-rmse:27764.1

[0]	validation_0-rmse:170220
Will train until validation_0-rmse hasn't improved in 50 rounds.
[100]	validation_0-rmse:27586
[200]	validation_0-rmse:26579.5
[300]	v

In [18]:
print('5 folds xgboost performance {}'.format(rmse(y_test,pred)))

5 folds xgboost performance 24030.454858888552


**7** L'objectif de cette question est d'implémenter un stacking simple.
-  1 - Commencer par apprendre des modèles sur le jeu d'apprentissage.
-  2 - Stocker les prédictions sur le jeu de validation dans une dataframe 'design_couche1', faites de même pour le jeu de test : 'test_couche1'.
-  3 - Apprennez un nouveau modèle de votre choix sur la nouvelle matrice d'apprentissage
-  4 - Evaluez les performances sur la nouvelle matrice de test.

In [19]:
m1 = RandomForestRegressor(n_estimators = 300).fit(X_train,y_train)
m2 = XGBRegressor(n_estimators = 300).fit(X_train,y_train)
m3 = GradientBoostingRegressor(n_estimators = 300).fit(X_train,y_train)
m4 = Ridge().fit(X_train,y_train)

In [20]:
design_couche1 = pd.DataFrame()
design_couche1['m1'] = m1.predict(X_val)
design_couche1['m2'] = m1.predict(X_val)
design_couche1['m3'] = m3.predict(X_val)
design_couche1['m4'] = m4.predict(X_val)
design_couche1.head()

,m1,m2,m3,m4
0,265395.626667,265395.626667,268983.652464,242459.096923
1,235550.946667,235550.946667,224910.936162,235770.689831
2,150989.683333,150989.683333,142216.118563,148715.756147
3,152945.690000,152945.690000,157305.405794,152669.094489
4,168790.920000,168790.920000,177003.893372,176054.153724


In [21]:
test_couche1 = pd.DataFrame()
test_couche1['m1'] = m1.predict(X_test)
test_couche1['m2'] = m1.predict(X_test)
test_couche1['m3'] = m3.predict(X_test)
test_couche1['m4'] = m4.predict(X_test)
test_couche1.head()

,m1,m2,m3,m4
0,150465.803333,150465.803333,140001.976213,147510.976904
1,167881.260000,167881.260000,170056.446058,163824.931601
2,116457.360000,116457.360000,100645.492464,92380.755873
3,165391.066667,165391.066667,155471.393358,138227.256277
4,188710.123333,188710.123333,191950.683037,231597.483871


In [22]:
from sklearn.linear_model import LinearRegression
modele_couche2 = LinearRegression()
modele_couche2.fit(design_couche1, y_val)
print('Performance stacking simple avec une LR {}'.format(rmse(y_test,modele_couche2.predict(test_couche1))))

Performance stacking simple avec une LR 23164.947057861402
